# Models
In this notebook we will generate several different models using the already engineered features. We will then evaluate the models using cross validation and select the best model to use for our predictions.

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [69]:
GEO = 'texas_florida'
df = pd.read_csv(f'../../data/processed/features_{GEO}.csv')
df.index = df["review_id"]
df.drop(columns=["review_id"], inplace=True)

print(df.shape)
df.head()

(1629, 101)


,tsne2_comments_2D_0,tsne2_comments_2D_1,tsne3_comments_3D_0,tsne3_comments_3D_1,tsne3_comments_3D_2,tsne2_description_2D_0,tsne2_description_2D_1,tsne3_description_3D_0,tsne3_description_3D_1,tsne3_description_3D_2,...,amenityall_dishwasher,amenityall_silverware,amenityall_bathtub,amenityall_lockbox,amenityall_crib,amenityall_elevator,amenityall_gym,amenityall_oven,amenityall_toilet,label
review_id,,,,,,,,,,,,,,,,,,,,,
83097,36.76,-100.08,7.22,-41.55,2.67,36.76,-100.08,7.22,-41.55,2.67,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,good
133337,36.76,-100.08,10.17,-41.86,-2.04,36.76,-100.08,10.17,-41.86,-2.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,good
150928,32.36,-99.52,11.88,-44.53,2.47,32.36,-99.52,11.88,-44.53,2.47,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,good
2706775,32.36,-99.52,11.88,-44.53,2.47,32.36,-99.52,11.88,-44.53,2.47,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,good
8602878,32.36,-99.52,11.88,-44.53,2.47,32.36,-99.52,11.88,-44.53,2.47,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,good


In [70]:
df.label.value_counts()

good     1320
mbad      118
mgood     105
bad        84
Name: label, dtype: int64

Drop NA labels... THANKS Charlie :/

In [71]:
df = df.dropna()

The data is extremely imbalanced. We can use the `imbalanced-learn` Python library, which has several implementations of resampling methods to deal with this problem. We will use the `SMOTE` method, which oversamples the minority class.

In [72]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(df.drop(columns=["label"]), df["label"])

print("Balanced labels:")
y.value_counts()

Balanced labels:


good     1320
mgood    1320
mbad     1320
bad      1320
Name: label, dtype: int64

## Testing out different feature combinations
We want to try the different combinations of features to see which ones work best.

In [79]:
import itertools

feature_sets = [
    "tsne2_comments_2D_",
    "tsne3_comments_3D_",
    "tsne2_description_2D_",
    "tsne3_description_3D_",
    "3gram09_",
    "3gram10_",
    "amenity005_",
    "amenityall_"
]

# get all combinations of feature sets of all lengths
feature_sets_combinations = []
for i in range(1, len(feature_sets) + 1):
    feature_sets_combinations += [list(x) for x in itertools.combinations(feature_sets, i)]

print(f"Number of feature sets combinations: {len(feature_sets_combinations)}")

Number of feature sets combinations: 255


In [80]:
from lazypredict.Supervised import LazyClassifier

top_models = {}
best_model = None
best_accuracy = -1
best_feature_set = None

num_of_feature_sets_combinations = len(feature_sets_combinations)

for i, feature_sets_combination in enumerate(feature_sets_combinations):

    print(f"Testing feature sets combination {i + 1} of {num_of_feature_sets_combinations} ({feature_sets_combination})")

    # get column subset
    df_subset_columns = [c for c in df.columns if any([c.startswith(f) for f in feature_sets_combination])]
    df_subset_columns.append("label")

    # oversample and get train/test split
    df_subset = df[df_subset_columns]
    X, y = oversample.fit_resample(df_subset.drop(columns=["label"]), df_subset["label"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # train model
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    top_models_for_subset = models.sort_values(by="Accuracy", ascending=False).head(10)

    top_model = models.iloc[0]
    accuracy = top_model["Accuracy"]
    name = top_model.name
    print(f"Top model for {feature_sets_combination}: {name}. Accuracy = {accuracy}")
    if top_model["Accuracy"] > best_accuracy:
        best_accuracy = top_model["Accuracy"]
        best_model = top_model
        best_feature_set = feature_sets_combination
        print(f"New best model found! Accuracy = {accuracy}")

    # save results
    top_models[",".join(feature_sets_combination)] = top_models_for_subset

    print()


print(f"Best model: {best_model.name} with feature set {best_feature_set}\nAccuracy = {best_accuracy}")

import pickle
# save top_models
with open(f"../../data/processed/top_models_{GEO}.pickle", "wb") as f:
    pickle.dump(top_models, f)

Testing feature sets combination 1 of 255 (['tsne2_comments_2D_'])


100%|██████████| 29/29 [00:05<00:00,  5.50it/s]


Top model for ['tsne2_comments_2D_']: ExtraTreesClassifier. Accuracy = 0.696969696969697
New best model found! Accuracy = 0.696969696969697

Testing feature sets combination 2 of 255 (['tsne3_comments_3D_'])


100%|██████████| 29/29 [00:05<00:00,  5.50it/s]


Top model for ['tsne3_comments_3D_']: ExtraTreesClassifier. Accuracy = 0.7973484848484849
New best model found! Accuracy = 0.7973484848484849

Testing feature sets combination 3 of 255 (['tsne2_description_2D_'])


...

Top model for ['tsne2_comments_2D_', 'tsne2_description_2D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenity005_', 'amenityall_']: ExtraTreesClassifier. Accuracy = 0.9119318181818182

Testing feature sets combination 254 of 255 (['tsne3_comments_3D_', 'tsne2_description_2D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenity005_', 'amenityall_'])


100%|██████████| 29/29 [00:27<00:00,  1.07it/s]


Top model for ['tsne3_comments_3D_', 'tsne2_description_2D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenity005_', 'amenityall_']: ExtraTreesClassifier. Accuracy = 0.9261363636363636

Testing feature sets combination 255 of 255 (['tsne2_comments_2D_', 'tsne3_comments_3D_', 'tsne2_description_2D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenity005_', 'amenityall_'])


100%|██████████| 29/29 [00:29<00:00,  1.02s/it]

Top model for ['tsne2_comments_2D_', 'tsne3_comments_3D_', 'tsne2_description_2D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenity005_', 'amenityall_']: ExtraTreesClassifier. Accuracy = 0.9176136363636364

Best model: ExtraTreesClassifier with feature set ['tsne3_comments_3D_', 'tsne3_description_3D_', '3gram09_', '3gram10_', 'amenityall_']
Accuracy = 0.9356060606060606


In [ ]:
report = []

# iterate over keys and values
for key, value in top_models.items():
    top_model_name = value.iloc[0].name
    top_model_accuracy = value.iloc[0]["Accuracy"]

    report.append({
        "feature_set": key,
        "top_model_name": top_model_name,
        "top_model_accuracy": top_model_accuracy
    })

top_10 = pd.DataFrame(report).sort_values(by="top_model_accuracy", ascending=False).head(10)
top_10

,feature_set,top_model_name,top_model_accuracy
208,"tsne3_comments_3D_,tsne3_description_3D_,3gram...",ExtraTreesClassifier,0.94
230,"tsne2_comments_2D_,tsne3_comments_3D_,tsne3_de...",ExtraTreesClassifier,0.93
179,"tsne2_comments_2D_,tsne3_comments_3D_,3gram09_...",ExtraTreesClassifier,0.93
149,"tsne2_description_2D_,tsne3_description_3D_,3g...",ExtraTreesClassifier,0.93
84,"tsne3_description_3D_,3gram09_,amenityall_",ExtraTreesClassifier,0.93
253,"tsne3_comments_3D_,tsne2_description_2D_,tsne3...",ExtraTreesClassifier,0.93
199,"tsne3_comments_3D_,tsne2_description_2D_,tsne3...",ExtraTreesClassifier,0.93
141,"tsne3_comments_3D_,tsne3_description_3D_,3gram...",ExtraTreesClassifier,0.93
229,"tsne2_comments_2D_,tsne3_comments_3D_,tsne3_de...",ExtraTreesClassifier,0.93
105,"tsne2_comments_2D_,tsne3_comments_3D_,3gram10_...",ExtraTreesClassifier,0.93


Let's grab the best subset, and hyperparameter tune the best model!

In [96]:
best_feature_set = top_10.reset_index().iloc[0].feature_set

columns_in_best_feature_set = [c for c in df.columns if any([c.startswith(f) for f in best_feature_set.split(",")])]
columns_in_best_feature_set.append("label")

df_best_subset = df[columns_in_best_feature_set]


In [97]:
# load ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

# apply GridSearchCV to find the best parameters
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['auto', 'sqrt', 'log2'],
}

clf = GridSearchCV(ExtraTreesClassifier(), params, cv=5, n_jobs=-1, verbose=1)

# oversample and get train/test split
X, y = oversample.fit_resample(df_best_subset.drop(columns=["label"]), df_best_subset["label"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)
best_clf = clf.best_estimator_

# print the best parameters
print("Best parameters:")
print(clf.best_params_)

# predict
y_pred = best_clf.predict(X_test)

# evaluate
from sklearn.metrics import accuracy_score

print("Accuracy:", round(accuracy_score(y_test, y_pred), 2))

# plot a confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
cm = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()

Fitting 5 folds for each of 108 candidates, totalling 540 fits
